1. pick one bug with all its explanations
2. manually write a merged explanation
3. find a metric to evaluate the quality of the merged explanation
4. set a threshold
5. try for different numbers of raw explanations merged
6. qualitatively evaluate